In [6]:
# Base Data Science snippet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tqdm import tqdm_notebook
import sys
sys.path.append("../")
from delivery import *
from delivery import run_episode, run_n_episodes
from Env import DeliveryEnvironment
from qagent import QAgent
import pymongo
from route_db import Storage
import datetime


%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
print(np.arange(10)+ 10*0)
print(np.arange(10)+ 10*1)

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]


### Points of interest and config

In [2]:
n_points = 20
max_box = 10
n_stops = 10
n_samples = 10**6

pois = np.random.rand(n_points, 2) * max_box
print(pois)

[[3.77674084 2.5109965 ]
 [8.66220256 7.70810199]
 [0.47831518 1.627469  ]
 [1.08928921 0.69059554]
 [3.10515836 3.8800278 ]
 [5.59265152 0.91461273]
 [4.87644423 7.84721133]
 [1.39131812 3.70706711]
 [5.19676547 6.03717161]
 [4.72979396 3.00128393]
 [2.83376705 5.58364542]
 [3.72972349 4.82285479]
 [3.83092676 4.29235975]
 [7.73196278 4.87796473]
 [7.24148408 1.19382718]
 [5.53968399 1.53237755]
 [8.99998693 2.28254795]
 [8.99338134 5.73676345]
 [0.17080524 1.53399802]
 [0.54760521 0.92570971]]


### Simple TSP route gen

In [38]:
def gen_samples(db, n_points, max_box, n_stops, n_samples, pois=None, n_classes=1):
    if pois is None:
        pois = np.random.rand(n_classes, n_points, 2) * max_box 
        # print(pois)
    for k in range(pois.shape[0]):
        for i in range(n_samples):
            if i % (n_samples/10) == 0:
                print("Test gen {} out of {}".format(i+1, n_samples))
            task_indices = np.random.choice(n_points, n_stops, replace=False)
            reverse_task_ind = np.column_stack((np.arange(n_stops), task_indices))
            task_locations = pois[k,task_indices, :]
            # no does_task_exist check
            env = DeliveryEnvironment(n_stops = n_stops, method = "distance", poi = task_locations)
            agent = QAgent(nA=env.action_space, nS=env.observation_space, alpha=0.8, gamma=0.8, eps_start=1, eps_decay=0.95, eps_min=0.01, sars='max')
            env, agent, reward = gen_n_episodes(env,agent, n_episodes=500)
            #env.render()
            route_indices = env.stops # need a getter?
            # global route is for class 0: poi 0-n_points-1, for class 1: poi n-points - 2*poins  
            global_route_indices = reverse_task_ind[route_indices,1] + (n_points * k)
            task_indices += (n_points * k) 
            # print("k glb ind, task ind", k, global_route_indices, task_indices)
            # add to the database
            route_dict = {"class": str(k), 
                          "input_graph": ','.join(map(str, task_locations.tolist())), 
                          "input_graph_ind": ','.join(map(str, task_indices.tolist())), 
                          "route": ','.join(map(str, global_route_indices.tolist())),
                          "reward": reward}
            res = db.insert_route(route_dict)
    

    print('done')

np.random.seed(3)
dt = datetime.datetime.now().strftime("%d-%H-%M-%S")
# enable database 
st_table = "TSP10Gen"+dt
print("Table: ", st_table)
db = Storage(st_table)

gen_samples(db, n_points=20, max_box=10, n_stops=10, n_samples=10000, n_classes=10)

Table:  TSP10Gen29-23-44-10
Test gen 1 out of 10000
Test gen 1001 out of 10000
Test gen 2001 out of 10000
Test gen 3001 out of 10000
Test gen 4001 out of 10000
Test gen 5001 out of 10000
Test gen 6001 out of 10000
Test gen 7001 out of 10000
Test gen 8001 out of 10000
Test gen 9001 out of 10000
Test gen 1 out of 10000
Test gen 1001 out of 10000
Test gen 2001 out of 10000
Test gen 3001 out of 10000
Test gen 4001 out of 10000
Test gen 5001 out of 10000
Test gen 6001 out of 10000
Test gen 7001 out of 10000
Test gen 8001 out of 10000
Test gen 9001 out of 10000
Test gen 1 out of 10000
Test gen 1001 out of 10000
Test gen 2001 out of 10000
Test gen 3001 out of 10000
Test gen 4001 out of 10000
Test gen 5001 out of 10000
Test gen 6001 out of 10000
Test gen 7001 out of 10000
Test gen 8001 out of 10000
Test gen 9001 out of 10000
Test gen 1 out of 10000
Test gen 1001 out of 10000
Test gen 2001 out of 10000
Test gen 3001 out of 10000
Test gen 4001 out of 10000
Test gen 5001 out of 10000
Test gen 600

In [37]:
db.print_table(limit=100)

{'_id': ObjectId('5d90d1fa31799063eb8f2b97'), 'class': '0', 'input_graph': '[8.617342119460838, 1.6058884690433972],[3.099555150295018, 9.181228573763379],[3.7597406135258584, 0.25870968454200116],[7.73443503716806, 7.030253918901547],[3.6696651142525916, 9.980586883249735],[3.100533937376402, 1.8305695344757211],[3.288688912737375, 5.149750727610351],[3.1753886912783247, 9.163955567387527],[4.90046810945453, 9.433116616354832],[6.300250033501631, 9.916431682678402]', 'input_graph_ind': '18,16,0,5,17,15,14,12,1,9', 'route': '12,16,17,1,9,5,14,15,0,18', 'reward': -22.00748575537659}
{'_id': ObjectId('5d90d1fa31799063eb8f2b98'), 'class': '0', 'input_graph': '[3.288688912737375, 5.149750727610351],[7.73443503716806, 7.030253918901547],[3.292047351062195, 6.1043162797802495],[3.6696651142525916, 9.980586883249735],[4.90046810945453, 9.433116616354832],[8.617342119460838, 1.6058884690433972],[1.6164112676738307, 6.526353509560687],[5.359599616856315, 3.9901656032275135],[7.906002906878729, 

In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)
######################################################################
arr = [19,20,2,4,5]
embeds = nn.Embedding(25, 64) # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor(arr[0], dtype=torch.long)
lookup_tensor1 = torch.tensor(arr[0], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
hello_embed1 = embeds(lookup_tensor1)
print(hello_embed, hello_embed1)

tensor([-0.2844,  0.6525,  1.4875,  0.4488, -1.1866, -0.4145,  0.0546,  0.7364,
         2.1369, -0.7117, -1.1548,  0.0066,  0.2640,  0.2466,  0.2442,  1.0261,
        -0.8111,  1.5332, -1.3854, -0.9218, -0.5595, -0.5135, -1.0420,  0.6778,
         0.1507,  0.4481,  0.1001,  1.5422,  0.9785,  1.6087,  0.8545,  0.7296,
         0.6838,  0.6625,  0.9380, -2.0800,  1.4300,  1.4052, -0.8558, -0.5066,
         1.0196, -0.9315, -0.1972,  1.0541, -1.2512, -0.2749,  0.1635,  1.1843,
         1.1478, -0.7313, -0.2146, -1.0845, -0.1757,  0.3241, -1.1501, -1.0863,
        -0.4153,  1.1012, -0.1722,  0.2987,  0.8592,  1.8661, -0.0838, -1.5336],
       grad_fn=<EmbeddingBackward>) tensor([-0.2844,  0.6525,  1.4875,  0.4488, -1.1866, -0.4145,  0.0546,  0.7364,
         2.1369, -0.7117, -1.1548,  0.0066,  0.2640,  0.2466,  0.2442,  1.0261,
        -0.8111,  1.5332, -1.3854, -0.9218, -0.5595, -0.5135, -1.0420,  0.6778,
         0.1507,  0.4481,  0.1001,  1.5422,  0.9785,  1.6087,  0.8545,  0.7296,
   